In [1]:
from model import client

response = client.chat.completions.create(
    model='glm-4-0520',
    messages=[{"role": "user", "content": "hello glm!"}],
    temperature=0.2,
    max_tokens=4096,
    stream=False
)

print(response.choices[0].message.content)

Hello 👋! I'm ChatGLM（智谱清言）, the artificial intelligence assistant, nice to meet you. Feel free to ask me any questions.


In [ ]:
from langchain_experimental.tools.python.tool import PythonAstREPLTool
import pandas as pd

# python_repl_ast
df = pd.read_excel("titanic.xlsx")
python_repl_ast = PythonAstREPLTool(locals={'df':df})

# 工具描述与字典
tools_description = [{
    "type": "function",
    "function": {
        "name": "python_repl_ast",
        "description": "一个Python shell。使用这个工具来执行Python命令。输入应该是一个有效的Python命令。在使用这个工具时，有时候输出会被省略——在使用它回答之前，请确保它看起来没有被省略。",
        "parameters": {
            "type": "object",
            "properties": {
                "tool_input": {
                    "description": "一个有效的Python命令",
                    "type": "str"
                }
            },
            "required": [
                "tool_input"
            ]
        }
    }
}]

tools_map = {
    "python_repl_ast": python_repl_ast
}

tools = {
    "description": tools_description,
    "map": tools_map
}

In [2]:
from tools import tools_description, tools_map
import json

def predict(messages):
    response = client.chat.completions.create(
        model='glm-4-0520',
        messages=messages,
        temperature=0,
        tools = tools_description,
    )

    return response.choices[0].message.model_dump()

In [3]:
prompt = '使用python找出50以内的质数'
messages = [{'role':'user', 'content':prompt}]

response_messages = predict(messages)
print(response_messages)

messages.append(response_messages)

{'content': '', 'role': 'assistant', 'tool_calls': [{'id': 'call_2024082911361346d843553d984253', 'function': {'arguments': '{"tool_input": "primes = []\\nfor num in range(2, 51):\\n    is_prime = True\\n    for i in range(2, int(num ** 0.5) + 1):\\n        if num % i == 0:\\n            is_prime = False\\n            break\\n    if is_prime:\\n        primes.append(num)\\nprimes"}', 'name': 'python_repl_ast'}, 'type': 'function', 'index': 0}]}


In [4]:
tools_calls_obj = response_messages['tool_calls']
tools_calls_obj

[{'id': 'call_2024082911361346d843553d984253',
  'function': {'arguments': '{"tool_input": "primes = []\\nfor num in range(2, 51):\\n    is_prime = True\\n    for i in range(2, int(num ** 0.5) + 1):\\n        if num % i == 0:\\n            is_prime = False\\n            break\\n    if is_prime:\\n        primes.append(num)\\nprimes"}',
   'name': 'python_repl_ast'},
  'type': 'function',
  'index': 0}]

In [5]:
def func_call(tools_calls_obj):
    function_obj = tools_calls_obj[0]['function']

    func = tools_map[function_obj['name']]
    args = json.loads(function_obj['arguments'])

    return json.dumps(func(**args))

In [6]:
func_call(tools_calls_obj)

/tmp/ipykernel_17467/1909064332.py:7: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use invoke instead.
  return json.dumps(func(**args))


'[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]'

In [7]:
messages.append({
    "role":"tool",
    "content": func_call(tools_calls_obj),
    "tool_call_id": tools_calls_obj[0]["id"]
})

messages

[{'role': 'user', 'content': '使用python找出50以内的质数'},
 {'content': '',
  'role': 'assistant',
  'tool_calls': [{'id': 'call_2024082911361346d843553d984253',
    'function': {'arguments': '{"tool_input": "primes = []\\nfor num in range(2, 51):\\n    is_prime = True\\n    for i in range(2, int(num ** 0.5) + 1):\\n        if num % i == 0:\\n            is_prime = False\\n            break\\n    if is_prime:\\n        primes.append(num)\\nprimes"}',
     'name': 'python_repl_ast'},
    'type': 'function',
    'index': 0}]},
 {'role': 'tool',
  'content': '[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]',
  'tool_call_id': 'call_2024082911361346d843553d984253'}]

In [8]:
predict(messages)

{'content': '50以内的质数有：2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47。',
 'role': 'assistant',
 'tool_calls': None}

In [1]:
from pathlib import Path
from string import Template
from model import client
import json

class Agent:
    def __init__(self, tools, prompt_template_name=None):
        self.messages = []
        self.tools_description = tools['description']
        self.tools_map = tools['map']

        if prompt_template_name:
            self.template = Template(Path(f'templates/{prompt_template_name}.txt').read_text())
        else:
            self.template = None


    def prompt_template_format(self, **kwargs):
        return self.template.substitute(**kwargs)
    
    def predict(self, messages):
        response = client.chat.completions.create(
            model='glm-4-0520',
            messages=messages,
            temperature=0,
            tools = self.tools_description,
        )

        return response.choices[0].message.model_dump()

    def func_call(self, tools_calls_obj):
        function_obj = tools_calls_obj[0]['function']

        func = self.tools_map[function_obj['name']]
        args = json.loads(function_obj['arguments'])

        return json.dumps(func(**args))
    
    def __call__(self, input):

        # 如果模板存在，根据模板格式化输入
        if self.template:
            input = self.prompt_template_format(input=input)

        # 第一次调用大模型
        self.messages.append({'role': 'user', 'content': input})
        response_messages = self.predict(self.messages)
        self.messages.append(response_messages)

        # 当需要调用函数时，执行func_call，并将返回值添加到message中，再一次调用大模型
        while ('tool_calls' in response_messages) and (response_messages['tool_calls'] is not None):
            tools_calls_obj = response_messages['tool_calls']
            self.messages.append({
                "role":"tool",
                "content": self.func_call(tools_calls_obj),
                "tool_call_id": tools_calls_obj[0]["id"]
            })
            response_messages = self.predict(self.messages)

        return response_messages['content']

In [2]:
from tools import tools

agent = Agent(tools)

In [4]:
print(agent('使用python设计程序，回答“小明今年4岁，身高100cm。如果每年的身高增加10cm，那么小明80岁时的身高是多少？”，并解释计算过程'))

小明今年4岁，身高100cm。如果每年的身高增加10cm，那么我们可以通过以下计算过程来得出小明80岁时的身高：

1. 小明当前的年龄是4岁。
2. 小明当前的身高是100cm。
3. 每年身高增加10cm。
4. 小明到80岁时，还需要过80 - 4 = 76年。
5. 在这76年里，小明的身高将增加76 * 10cm = 760cm。
6. 因此，小明80岁时的身高将是初始身高加上这76年的增长，即100cm + 760cm = 860cm。

所以，小明80岁时的身高是860cm。
